In [1]:
pip install tensorflow keras


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


In [10]:
train_dir = "C:/Users/Adish/OneDrive/Desktop/SMARTINTERNZ/vitamin dataset/train_data"
test_dir = "C:/Users/Adish/OneDrive/Desktop/SMARTINTERNZ/vitamin dataset/test_data"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=15,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=15,
    class_mode='categorical'
)


Found 8969 images belonging to 5 classes.
Found 224 images belonging to 5 classes.


In [11]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # Assuming you have 5 classes (Vitamin A, B, C, D, E)

model = Model(inputs=base_model.input, outputs=predictions)




In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=10  
)


Epoch 1/10


D:\New folder\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


280/280 ━━━━━━━━━━━━━━━━━━━━ 3457s 12s/step - accuracy: 0.4049 - loss: 1.4998 - val_accuracy: 0.6116 - val_loss: 1.0594
Epoch 2/10
  1/280 ━━━━━━━━━━━━━━━━━━━━ 1:19:46 17s/step - accuracy: 0.3125 - loss: 1.3318

D:\New folder\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')


In [ ]:
model.save('vitamin_prediction_model.h5')


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg19 import preprocess_input


In [ ]:
img_path = 'path_to_your_image.jpg'  
img = image.load_img(img_path, target_size=(224, 224))  # Load the image
img_array = image.img_to_array(img)  # Convert the image to an array
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the model's input shape
img_array = preprocess_input(img_array)  # Preprocess the image


In [ ]:
predictions = model.predict(img_array)  # Predict the class of the image
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Get the index of the predicted class

# To map the predicted class index to class labels, you need the class indices
class_indices = {'vitamin A': 0, 'vitamin B': 1, 'vitamin C': 2, 'vitamin D': 3, 'vitamin E': 4}  # Example mapping
class_labels = list(class_indices.keys())
predicted_class = class_labels[predicted_class_index]

print(f'The predicted vitamin type is: {predicted_class}')
